In [1]:
# !pip install SQLAlchemy
# !pip install pyspark
# !pip install jupysql duckdb duckdb-engine
# !pip install delta-spark
# !pip install py4j
# !pip install -q findspark
# conda install -c conda-forge submitit 
# conda install corn, nbconvert
# !pip install graphviz
# !pip install hydra-core
# !pip install hydra-submitit-launcher
# !pip install submitit
# !pip install covalent-slurm-plugin
# !pip install ipynb-py-convert

In [2]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [3]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

# import submitit

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)  # Increase to 1000 or more as needed
spark.conf.set("spark.sql.execution.arrow.enabled", "false")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/01 01:35:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/01 01:35:10 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [4]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")
df_population = spark.sql("SELECT * FROM population")

In [5]:
headerList = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [6]:
def whether_thisVIN_data_already_generated(thisVIN): 
    df_VINs_data = spark.sql("SELECT * FROM VINs_data")
    df_VINs_data = df_VINs_data.toDF("VIN", "TOTAL_ROWS")
    df_VINs_data_p = df_VINs_data.toPandas()
    for index, row in df_VINs_data_p.iterrows():
        if row['VIN'] == thisVIN:
          return 1
    return 0

# Function to submit a SLURM job and return the job ID
def submit_job1(script_path, thisVIN, account='vuh14_dibbs_sc', partition='sla-prio'):
    submit_command = f"sbatch --account={account} --partition={partition} {script_path} {thisVIN}"
    print("The submit command for the submitted job is: ")
    print(submit_command)
    result = subprocess.run(submit_command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        job_id = re.findall(r'\d+', result.stdout)[0]
        print(f"Job {job_id} submitted successfully.")
        return job_id
    else:
        print("Job submission failed.")
        return None

# Function to submit a SLURM job and return the job ID
def submit_job2(script_path, thisVIN, calendar_day_from_where_it_left_off, account='vuh14_dibbs_sc', partition='sla-prio'):
    submit_command = f"sbatch --account={account} --partition={partition} {script_path} {thisVIN} {calendar_day_from_where_it_left_off}"
    print("The submit command for the submitted job is: ")
    print(submit_command)
    result = subprocess.run(submit_command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        job_id = re.findall(r'\d+', result.stdout)[0]
        print(f"Job {job_id} submitted successfully.")
        return job_id
    else:
        print("Job submission failed.")
        return None
 
# Function to check the status of multiple jobs
def check_jobs_status(job_ids):
    jobs_running = set(job_ids)
    while jobs_running:
        for job_id in list(jobs_running):
            status_command = f"squeue -j {job_id}"
            result = subprocess.run(status_command, shell=True, capture_output=True, text=True)
            if not result.stdout:  # If the job is no longer in the queue
                print(f"Job {job_id} completed.")
                jobs_running.remove(job_id)
        if jobs_running:
            print("Some jobs are still running...")
            print(jobs_running)
            t.sleep(1)  # Check every 5 seconds

# Function to retrieve job outputs
def get_jobs_output(output_files):
    for output_file in output_files:
        try:
            with open(output_file, "r") as file:
                output = file.read()
            print(f"Output from {output_file}:")
            print(output)
        except FileNotFoundError:
            print(f"Output file {output_file} not found.")
            
            
def update_VINs_data_total_rows_from_resulted_data(vins_data_path, resulted_data_path, headerList):
    import pandas as pd

    # Step 1: Read VINs_data.csv
    df_vins = pd.read_csv(vins_data_path, names=["VIN", "TOTAL_ROWS"], dtype=str, header=None)

    # Step 2: Read resultedData.csv with headerList, skipping bad lines
    df_resulted = pd.read_csv(resulted_data_path, names=headerList, header=None, on_bad_lines='skip')

    # Step 3: Ensure calendar_day is numeric
    df_resulted["calendar_day"] = pd.to_numeric(df_resulted["calendar_day"], errors='coerce')

    # Step 4: Filter and update VINs
    updated_rows = []

    for index, row in df_vins.iterrows():
        vin = row['VIN']
        matching_rows = df_resulted[df_resulted['VIN'] == vin]

        if not matching_rows.empty:
            max_day = matching_rows['calendar_day'].max()
            print(f" VIN: {vin} → Max calendar_day: {max_day}, Updating TOTAL_ROWS to: {int(max_day)}")
            updated_rows.append([vin, int(max_day)])
        else:
            print(f" VIN: {vin} → No match in resultedData → Setting TOTAL_ROWS to 0")
            updated_rows.append([vin, 0])  # Mark for later removal

    # Step 5: Create DataFrame and remove TOTAL_ROWS == 0
    updated_df = pd.DataFrame(updated_rows, columns=["VIN", "TOTAL_ROWS"])
    updated_df = updated_df[updated_df["TOTAL_ROWS"] != 0]

    # Step 6: Overwrite VINs_data.csv
    updated_df.to_csv(vins_data_path, index=False, header=False)

    print(f"\n VINs_data.csv updated. Final row count (excluding TOTAL_ROWS=0): {len(updated_df)}")
    
    
    
def is_valid_vin(vin: str) -> int:
    """
    Returns 1 if the VIN is valid, 0 if not.
    A valid VIN is exactly 17 characters long and does not contain I, O, or Q.
    """
    vin = vin.strip().upper()

    # Rule 1: VIN must be exactly 17 characters
    if len(vin) != 17:
        return 0

    # Rule 2: Must only contain A-H, J-N, P, R-Z, and 0-9 (excluding I, O, Q)
    if not re.match("^[A-HJ-NPR-Z0-9]{17}$", vin):
        return 0

    return 1  
    
    
def update_VINs_data_from_resulted_data(vins_data_path, resulted_data_path, headerList):

    # Columns: Keep selected + headers
    selected_features_from_population = ['VIN', 'ENGINE_SIZE', 'ENGINE_HP', 'VEH_TYPE'] + [s for s in df_population.columns if 'KOLA' in s]
    all_columns_names = selected_features_from_population + headerList

    #  Step 1: Load VINs_data.csv
    df_vins = pd.read_csv(VINs_data_file_path, names=["VIN", "TOTAL_ROWS"], dtype={"VIN": str, "TOTAL_ROWS": int}, header=None)

    #  Step 2: Load resultedData.csv (skip bad lines)
    df_resulted = pd.read_csv(resulted_data_path, names=all_columns_names, header=None, on_bad_lines='skip')

    #  Step 3: Convert calendar_day to numeric
    df_resulted["calendar_day"] = pd.to_numeric(df_resulted["calendar_day"], errors='coerce')

    #  Step 4: Group by VIN → max calendar_day
    grouped = df_resulted.groupby("VIN")["calendar_day"].max().reset_index()
    grouped.columns = ["VIN", "MAX_CALENDAR_DAY"]

    #  Step 5: Build dictionary of existing VINs
    vins_dict = dict(zip(df_vins["VIN"], df_vins["TOTAL_ROWS"]))

    #  Step 6: Update or add VINs
    updated_rows = []

    for _, row in grouped.iterrows():
        vin = row["VIN"]
        max_day_val = row["MAX_CALENDAR_DAY"]

        if(is_valid_vin(vin)==0):
            print(f" Skipping VIN: {vin} because it is invalid.")
            continue

        if pd.isna(max_day_val):
            print(f" Skipping VIN: {vin} → MAX_CALENDAR_DAY is NaN")
            continue

        max_day = int(max_day_val)

        if vin in vins_dict:
            print(f" Updating VIN: {vin} → TOTAL_ROWS: {vins_dict[vin]} → {max_day}")
        else:
            print(f" Adding new VIN: {vin} → TOTAL_ROWS: {max_day}")

        updated_rows.append([vin, max_day])

    #  Step 7: Add VINs from original not in grouped
    existing_vins_set = set(grouped["VIN"])
    for vin, total_rows in vins_dict.items():
        if vin not in existing_vins_set:
            updated_rows.append([vin, total_rows])
            print(f" Keeping existing VIN: {vin} → TOTAL_ROWS: {total_rows}")

    #  Step 8: Save updated VINs
    df_final = pd.DataFrame(updated_rows, columns=["VIN", "TOTAL_ROWS"])
    df_final.to_csv(VINs_data_file_path, index=False, header=False)

    print(f"\n VINs_data.csv successfully updated. Total VINs: {len(df_final)}")

In [ ]:
VINs_data_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv'
resulted_data_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv'

# update_VINs_data_total_rows_from_resulted_data(VINs_data_file_path, resulted_data_path, headerList)
update_VINs_data_from_resulted_data(VINs_data_file_path, resulted_data_path, headerList)


df_all_pop_VINs = spark.sql('select distinct VIN from population')
df_all_pop_VINs_p = df_all_pop_VINs.toPandas()

# Read the VINs_data.csv (with header row: VIN,TOTAL_ROWS)
df_generated_pop_VINs = pd.read_csv(
    VINs_data_file_path, sep=',', names=['VIN','TOTAL_ROWS'],
    header=None, dtype=str
)

print(f"size of df_all_pop_VINs={len(df_all_pop_VINs_p)}, df_generated_pop_VINs = {len(df_generated_pop_VINs)}")

# Only proceed if the population has more VINs than the generated set
if len(df_all_pop_VINs_p) > len(df_generated_pop_VINs):   

    # Let this be your overall job limit
    number_of_jobs = 2
    jobs_submitted = 0
    job_ids = []
    output_files = []

    try:
        if len(df_generated_pop_VINs) > 0:
            df_generated_pop_VINs['TOTAL_ROWS'] = df_generated_pop_VINs['TOTAL_ROWS'].astype(int)
            processed_VINs = set(df_generated_pop_VINs['VIN'].astype(str))
        else:
            processed_VINs = set()
    except FileNotFoundError:
        print("VINs_data.csv not found. Starting fresh.")
        processed_VINs = set()

    # --- 1) Process partially generated VINs (TOTAL_ROWS < 2556) --- The second condition is temporary
    df_incomplete = df_generated_pop_VINs[df_generated_pop_VINs['TOTAL_ROWS'].between(999, 2557)]

    
    
    for idx, row in df_incomplete.iterrows():
        # If we've hit the limit, no more submissions
        if jobs_submitted >= number_of_jobs:
            break

        thisVIN = row['VIN']
        rows_val = row['TOTAL_ROWS']
        print(f"VIN={thisVIN} has TOTAL_ROWS={rows_val}, which is < 2556 so partial...")

        # Example usage: pass the last day as param
        calendar_day_from_where_it_left_off = rows_val

        # Submit partial job
        df_filtered_population_for_this_VIN = df_all_pop_VINs.filter(f.col('VIN') == thisVIN)
        script_path = "/storage/home/yqf5148/work/volvoPennState/Jobs/submit_partiallyCalculatedJobDone.sh"
        output_file = f"/storage/home/yqf5148/work/volvoPennState/Jobs/log_test/result_{thisVIN}.txt"
        job_id = submit_job2(script_path, thisVIN, calendar_day_from_where_it_left_off)

        if job_id:
            job_ids.append(job_id)
            output_files.append(output_file)
            processed_VINs.add(thisVIN)

            # Overwrite VINs_data.csv entry with 0
            new_VIN_entry = pd.DataFrame([[thisVIN, 0]], columns=['VIN','TOTAL_ROWS'])
            new_VIN_entry.to_csv(VINs_data_file_path, mode='a', index=False, header=False)

            jobs_submitted += 1

    # If we've used up all jobs in partial VINs alone, we're done
    if jobs_submitted >= number_of_jobs:
        print(f"Hit the job limit ({number_of_jobs}) while processing partial VINs.")
        check_jobs_status(job_ids)
        get_jobs_output(output_files)
        sys.exit(0)  # Or just return if in a function

    # --- 2) Now do random selection for ungenerated VINs ---
    # The code below remains the same, but includes a check so we don’t exceed the limit.
    
    total_VINs = len(df_all_pop_VINs_p)
    print(f"total_VINs: {total_VINs}")

    while jobs_submitted < number_of_jobs and len(processed_VINs) < total_VINs:
        random_index = random.randint(0, total_VINs - 1)
        print(f"Random generated index in overall {total_VINs} VINs is {random_index}")
        row = df_all_pop_VINs_p.iloc[random_index]
        thisVIN = str(row['VIN'])

        if thisVIN in processed_VINs or whether_thisVIN_data_already_generated(thisVIN) == 1:
            print(f"This VIN = {thisVIN}'s data is already generated. Selecting a new one...")
            continue
        else:
            print(f"This VIN = {thisVIN}'s data is not generated.")
            df_filtered_population_for_this_VIN = df_all_pop_VINs.filter(f.col('VIN') == thisVIN)
            script_path = "/storage/home/yqf5148/work/volvoPennState/Jobs/submit.sh"
            output_file = f"/storage/home/yqf5148/work/volvoPennState/Jobs/log_test/result_{thisVIN}.txt"
            job_id = submit_job1(script_path, thisVIN)

            if job_id:
                job_ids.append(job_id)
                output_files.append(output_file)
                processed_VINs.add(thisVIN)

                # Append a new entry
                new_VIN_entry = pd.DataFrame([[thisVIN, 0]], columns=['VIN','TOTAL_ROWS'])
                new_VIN_entry.to_csv(VINs_data_file_path, mode='a', index=False, header=False)

                jobs_submitted += 1

            # Stop if we hit the limit
            if jobs_submitted >= number_of_jobs:
                print(f"Hit the job limit ({number_of_jobs}) while picking random VINs.")
                break

    # --- 3) Check job status & retrieve outputs ---
    check_jobs_status(job_ids)
    get_jobs_output(output_files)


/tmp/ipykernel_1065063/2802079158.py:131: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_resulted = pd.read_csv(resulted_data_path, names=all_columns_names, header=None, on_bad_lines='skip')


 Skipping VIN: -AGS because it is invalid.
 Skipping VIN: -ST because it is invalid.
 Skipping VIN: .0 because it is invalid.
 Skipping VIN: 0 because it is invalid.
 Skipping VIN: 0.0 because it is invalid.
 Skipping VIN: 1 because it is invalid.
 Updating VIN: 1M1AN07Y3GM021042 → TOTAL_ROWS: 414 → 419
 Updating VIN: 1M1AN2AY5LM001812 → TOTAL_ROWS: 2084 → 2084
 Updating VIN: 1M1AN2GY3KM006268 → TOTAL_ROWS: 29 → 29
 Updating VIN: 1M1AN3GY4LM019358 → TOTAL_ROWS: 29 → 29
 Updating VIN: 1M1AN4GY0MM022958 → TOTAL_ROWS: 1914 → 1914
 Updating VIN: 1M1AN4GY0MM022961 → TOTAL_ROWS: 2339 → 2339
 Updating VIN: 1M1AN4GY7KM008343 → TOTAL_ROWS: 779 → 789
 Updating VIN: 1M1AN4GY8MM021279 → TOTAL_ROWS: 2199 → 2199
 Updating VIN: 1M1AN4GYXLM018348 → TOTAL_ROWS: 29 → 29
 Updating VIN: 1M1AW01Y2FM006911 → TOTAL_ROWS: 1433 → 1433
 Updating VIN: 1M1AW01Y5JM010413 → TOTAL_ROWS: 24 → 24
 Updating VIN: 1M1AW02Y1FM050302 → TOTAL_ROWS: 29 → 29
 Updating VIN: 1M1AW02Y7FM051728 → TOTAL_ROWS: 764 → 764
 Updating V

size of df_all_pop_VINs=322178, df_generated_pop_VINs = 85
VIN=1M1AN2AY5LM001812 has TOTAL_ROWS=2084, which is < 2556 so partial...
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submit_partiallyCalculatedJobDone.sh 1M1AN2AY5LM001812 2084
Job 36084542 submitted successfully.
VIN=1M1AN4GY0MM022958 has TOTAL_ROWS=1914, which is < 2556 so partial...
The submit command for the submitted job is: 
sbatch --account=vuh14_dibbs_sc --partition=sla-prio /storage/home/yqf5148/work/volvoPennState/Jobs/submit_partiallyCalculatedJobDone.sh 1M1AN4GY0MM022958 1914
Job 36084543 submitted successfully.
Hit the job limit (2) while processing partial VINs.
Some jobs are still running...
{'36084542', '36084543'}
Some jobs are still running...
{'36084542', '36084543'}
Some jobs are still running...
{'36084542', '36084543'}
Some jobs are still running...
{'36084542', '36084543'}
Some jobs are still running...
{

In [ ]:
display(df_pop_VINs_p)